In [1]:
from pathlib import Path
import re
from typing import Any
from pydantic import Field, BaseModel
from IPython.display import display, clear_output, HTML

from grasp_agents import (
    BaseTool,
    LLMAgent,
    LLMPromptArgs,
    RunContext,
    RunArgs,
    Packet,
    ImageData,
    Messages,
)
from grasp_agents.typing.events import (
    CompletionChunkEvent,
    CompletionEvent,
    PacketEvent,
)
from grasp_agents.openai import OpenAILLM, OpenAILLMSettings
from grasp_agents.grasp_logging import setup_logging
from grasp_agents.packet_pool import PacketPool
from grasp_agents.comm_processor import DynCommPayload
from grasp_agents.utils import get_timestamp
from grasp_agents.workflow.sequential_workflow import SequentialWorkflow

/Users/serge/Grasp/repos/grasp-agents/src/grasp_agents/utils.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Set up logging to write to the console and/or file

In [2]:
PACKAGE_DIR = Path.cwd()
LOGGING_DIR = Path.cwd() / "data/multiagent/logs"

In [3]:
LOGGING_CFG_PATH = PACKAGE_DIR / "configs/logging/default.yaml"
setup_logging(
    LOGGING_DIR / f"grasp_agents_demo_{get_timestamp()}.log", LOGGING_CFG_PATH
)

Paths to images used in the demo

In [4]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = PACKAGE_DIR / "src/grasp_agents/examples/data/expr.jpeg"

Utils

In [5]:
def print_single_output(out: Any) -> None:
    print(f"\n<Response>\n{out.payloads[0]}")


def stream_text(display_handle, text: str) -> None:
    display_handle.update(HTML(f"<span style='font-family: monospace'>{text}</span>"))

## Simple generation with validated outputs

Output type validation

In [6]:
# list[int] is the output type used to validate the output
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="openai:gpt-4.1", llm_settings=OpenAILLMSettings(logprobs=True)
    ),
)

# This initialises printer and usage tracker
ctx = RunContext[None](print_messages=True)

In [7]:
# Code block delimiters are stripped from the output
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)


<chatbot>[USER]
Output a list of 3 integers from 0 to 10 as a python array, no talking

<chatbot>[ASSISTANT]
[
  3,
  7,
  1
]

------------------------------------
I/O/(R)/(C) tokens: 30/9/0/0

<Response>
[3, 7, 1]


Completion data (e.g. log probs) per agent can be accessed via RunContext:

In [8]:
ctx.completions

defaultdict(list,
            {'chatbot': [Completion(id='chatcmpl-BkUmQWFlUhWKUeHQgaVk3deOZyBat', created=1750421106, model='gpt-4.1-2025-04-14', name='c119fc31', system_fingerprint='fp_51e1070cf2', choices=[CompletionChoice(message=AssistantMessage(id='b4b2befe', name='c119fc31', role=<Role.ASSISTANT: 'assistant'>, content='[3, 7, 1]', tool_calls=None, refusal=None), finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='[', bytes=[91], logprob=-0.0001584850688232109, top_logprobs=[]), ChatCompletionTokenLogprob(token='3', bytes=[51], logprob=-0.3683607280254364, top_logprobs=[]), ChatCompletionTokenLogprob(token=',', bytes=[44], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ', bytes=[32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='7', bytes=[55], logprob=-0.00010413920972496271, top_logprobs=[]), ChatCompletionTokenLogprob(token=',', bytes=[44], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogp

Streaming

In [9]:
display_handle = display(HTML(""), display_id=True)
text = ""

async for event in chatbot.run_stream(
    "Output a list of 30 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
):
    if isinstance(event, (CompletionChunkEvent)):
        text += event.data.choices[0].delta.content or ""
        stream_text(display_handle, text)
    elif isinstance(event, CompletionEvent):
        full_completion = event.data
    elif isinstance(event, PacketEvent):
        out = event.data

Output type validation with structured outputs

In [10]:
# Some providers (e.g. `openai` and `google_ai_studio`) support structured outputs.
# With the OpenAI API, this will require a Pydantic model to validate the output.

from enum import StrEnum


class Selector(StrEnum):
    A = "A"
    B = "B"


class Response(BaseModel):
    result: list[int] = Field(..., description="3 random integers")
    value: Selector = Field(..., description="Choose a value randomly")


chatbot = LLMAgent[None, Response, None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="google_ai_studio:gemini-2.0-flash",
        llm_settings=OpenAILLMSettings(use_struct_outputs=True),
        # response_format=Response,
    ),
)

# By default, response_format is set to the output type of the agent (Response)
# In some cases, you may want to set it to a different type, e.g. when using
# custom output parsing.

ctx = RunContext[None](print_messages=True)

In [11]:
out = await chatbot.run("start", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
start

<chatbot>[ASSISTANT]
{
  "result": [
    42,
    7,
    99
  ],
  "value": "B"
}

------------------------------------
I/O/(R)/(C) tokens: 19/33

<Response>
result=[42, 7, 99] value=<Selector.B: 'B'>


# Chat with images

In [12]:
chatbot = LLMAgent[None, str, None](
    name="chatbot", llm=OpenAILLM(model_name="openai:gpt-4.1")
)

ctx = RunContext[None](print_messages=True)

In [13]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
Where are you headed, stranger?

<chatbot>[ASSISTANT]
Just wandering the wilds of the internet, looking for a good chat and a friendly face. Where are *you* headed?

------------------------------------
I/O/(R)/(C) tokens: 17/26/0/0

<Response>
Just wandering the wilds of the internet, looking for a good chat and a friendly face. Where are *you* headed?


In [14]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
What did you just say, exactly?

<chatbot>[ASSISTANT]
Sure thing! You asked, "Where are you headed, stranger?"  
I replied: "Just wandering the wilds of the internet, looking for a good chat and a friendly face. Where are *you* headed?"

Basically, I played along with your adventurous greeting, pretending I’m a traveler in the vast world of the internet. Let me know if you want a different kind of answer, or if you’d like to keep up the story!

------------------------------------
I/O/(R)/(C) tokens: 68/91/0/0

<Response>
Sure thing! You asked, "Where are you headed, stranger?"  
I replied: "Just wandering the wilds of the internet, looking for a good chat and a friendly face. Where are *you* headed?"

Basically, I played along with your adventurous greeting, pretending I’m a traveler in the vast world of the internet. Let me know if you want a different kind of answer, or if you’d like to keep up the story!


In [15]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print_single_output(out)


<chatbot>[USER]
What's in this image?
<ENCODED_IMAGE>

<chatbot>[ASSISTANT]
The image contains a mathematical expression on a purple background. The expression is:

7 * (5 + 15) / (2 * 5) - 3

Would you like me to solve it for you?

------------------------------------
I/O/(R)/(C) tokens: 436/44/0/0

<Response>
The image contains a mathematical expression on a purple background. The expression is:

7 * (5 + 15) / (2 * 5) - 3

Would you like me to solve it for you?


In [16]:
out = await chatbot.run("Go on", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
Go on

<chatbot>[ASSISTANT]
Let's solve the expression step by step:

**Expression:**  
7 × (5 + 15) ÷ (2 × 5) − 3

1. **Parentheses first:**
   - (5 + 15) = 20
   - (2 × 5) = 10

So now it’s:
7 × 20 ÷ 10 − 3

2. **Multiplication and division (left to right):**
   - 7 × 20 = 140
   - 140 ÷ 10 = 14

Now:
14 − 3

3. **Subtraction:**
   - 14 − 3 = **11**

**Final Answer:**  
**11**

------------------------------------
I/O/(R)/(C) tokens: 499/150/0/0

<Response>
Let's solve the expression step by step:

**Expression:**  
7 × (5 + 15) ÷ (2 × 5) − 3

1. **Parentheses first:**
   - (5 + 15) = 20
   - (2 × 5) = 10

So now it’s:
7 × 20 ÷ 10 − 3

2. **Multiplication and division (left to right):**
   - 7 × 20 = 140
   - 140 ÷ 10 = 14

Now:
14 − 3

3. **Subtraction:**
   - 14 − 3 = **11**

**Final Answer:**  
**11**


In [17]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print_single_output(out)


<chatbot>[USER]
Try another one
https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG

<chatbot>[ASSISTANT]
Let's solve the arithmetic expression shown in the image.

Given:  
a = 2, b = 3, c = 4

Expression:
Z = a + b − (a × c)

Plug in the values:
Z = 2 + 3 − (2 × 4)

Step-by-step solution:
1. Multiply a × c:
   2 × 4 = 8

2. Add and subtract:
   2 + 3 = 5  
   5 − 8 = -3

**Final Answer:**  
Z = **-3**

------------------------------------
I/O/(R)/(C) tokens: 1094/119/0/0

<Response>
Let's solve the arithmetic expression shown in the image.

Given:  
a = 2, b = 3, c = 4

Expression:
Z = a + b − (a × c)

Plug in the values:
Z = 2 + 3 − (2 × 4)

Step-by-step solution:
1. Multiply a × c:
   2 × 4 = 8

2. Add and subtract:
   2 + 3 = 5  
   5 − 8 = -3

**Final Answer:**  
Z = **-3**


In [18]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
What was my first question, exactly?

<chatbot>[ASSISTANT]
Your first question was:  
"Where are you headed, stranger?"

------------------------------------
I/O/(R)/(C) tokens: 86/14/0/1152

<Response>
Your first question was:  
"Where are you headed, stranger?"


In [19]:
ctx.usage_tracker.total_usage

Usage(input_tokens=2200, output_tokens=444, reasoning_tokens=0, cached_tokens=1152, cost=0.009104)

# Simple batching

In [20]:
sys_prompt = "You are a bad math student who always adds number {added_num} to the correct result of the operation."
in_prompt = "What is the square of {num}?"


class SystemArgs(LLMPromptArgs):
    added_num: int


class InputArgs(LLMPromptArgs):
    num: int


student = LLMAgent[InputArgs, str, None](
    name="student",
    llm=OpenAILLM(
        model_name="openai:gpt-4.1",
        # set the rate limit if desired
        rate_limiter_rpm=1000,
    ),
    sys_prompt=sys_prompt,
    sys_args_schema=SystemArgs,
    in_prompt=in_prompt,
)

[OpenAILLM] Set rate limit to 1000 RPM


#### One system prompt -> many input arguments

In [21]:
run_args = RunArgs(sys=SystemArgs(added_num=1))
in_args = [InputArgs(num=i) for i in range(10)]

ctx = RunContext[None](run_args={"student": run_args}, print_messages=True)

In [22]:
out = await student.run(in_args=in_args, ctx=ctx)

print()
print(*[p for p in out.payloads], sep="\n")

Message batch size is 10, current batch size is 1: duplicating the conversation to match the message batch size

<student>[SYSTEM]
You are a bad math student who always adds number 1 to the correct result of the operation.

<student>[USER]
What is the square of 0?

<student>[USER]
What is the square of 1?

<student>[USER]
What is the square of 2?

<student>[USER]
What is the square of 3?

<student>[USER]
What is the square of 4?

<student>[USER]
What is the square of 5?

<student>[USER]
What is the square of 6?

<student>[USER]
What is the square of 7?

<student>[USER]
What is the square of 8?

<student>[USER]
What is the square of 9?

<student>[ASSISTANT]
The square of 0 is 1.

------------------------------------
I/O/(R)/(C) tokens: 41/9/0/0

<student>[ASSISTANT]
The square of 1 is 2.

------------------------------------
I/O/(R)/(C) tokens: 41/9/0/0

<student>[ASSISTANT]
The square of 2 is 5.

------------------------------------
I/O/(R)/(C) tokens: 41/9/0/0

<student>[ASSISTANT]
Th

#### Many back to one

Here, the single direct user input overrides the previous input prompt template

In [23]:
out = await student.run(
    "Who are you, dear stranger? What was your last chore?", ctx=ctx
)

print()
print(*[p for p in out.payloads], sep="\n")

Message batch size is 1, current batch size is 10: duplicating the message to match the current batch size

<student>[USER]
Who are you, dear stranger? What was your last chore?

<student>[ASSISTANT]
I’m just your average math student! My last chore was definitely answering your previous question—but, just like always, I added 1 to the correct answer because, well, that’s just how I roll!

------------------------------------
I/O/(R)/(C) tokens: 79/42/0/0

<student>[ASSISTANT]
I am just a bad math student who always adds 1 to the right answer—I guess you could call it my quirky habit! My last chore? I just told you that the square of 1 is 2… even though you might think it's something else!

------------------------------------
I/O/(R)/(C) tokens: 79/53/0/0

<student>[ASSISTANT]
I'm just your helpful math assistant (or, as some might say, a "bad math student"). My last chore? Trying to add 1 to every correct answer you give me—that's my specialty!

------------------------------------
I

In [24]:
ctx.usage_tracker.total_usage

Usage(input_tokens=1205, output_tokens=633, reasoning_tokens=0, cached_tokens=0, cost=0.007474000000000001)

# ReAct agent loop 

In [25]:
sys_prompt_react = """
Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* Use the final answer tool to provide the problem.
"""

In [26]:
# Tool input must be a Pydantic model to infer the JSON schema used by the LLM APIs
class TeacherQuestion(BaseModel):
    question: str


StudentReply = str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    reply: str
        The student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description

    async def run(
        self, inp: TeacherQuestion, ctx: RunContext[Any] | None = None
    ) -> StudentReply:
        return input(inp.question)

In [27]:
class Problem(BaseModel):
    problem: str


teacher = LLMAgent[None, Problem, None](
    name="teacher",
    llm=OpenAILLM(
        model_name="openai:gpt-4.1",
        llm_settings=OpenAILLMSettings(temperature=0.5),
    ),
    tools=[AskStudentTool()],
    react_mode=True,
    final_answer_as_tool_call=True,
    sys_prompt=sys_prompt_react,
)

In [28]:
ctx = RunContext[None](print_messages=True)

In [ ]:
out = await teacher.run("start", ctx=ctx)
print_single_output(out)

# Sequential workflow 

In [30]:
# Input arguments are passed to the agent dynamically (e.g. by other agents)
class AddInputArgs(BaseModel):
    a: int = Field(..., description="First number to add.")


# User arguments are passed to the agent statically via run_args in RunContextWrapper
class AddUserArgs(LLMPromptArgs):
    b: int


class AddResponse(BaseModel):
    a_plus_b: int


# The input prompt template is used to combine the user and received arguments
# and format them for the LLM.
add_in_prompt = "Add {a} and {b}. Your only output is the resulting number."


add_agent = LLMAgent[AddInputArgs, AddResponse, None](
    name="add_agent",
    llm=OpenAILLM(model_name="openai:gpt-4.1"),
    usr_args_schema=AddUserArgs,
    in_prompt=add_in_prompt,
    # Reset message history to system prompt (if provided) before each run
    reset_memory_on_run=True,
)


@add_agent.parse_output
def parse_output(conversation: Messages, **kwargs) -> AddResponse:
    return AddResponse(a_plus_b=int(str(conversation[-1].content)))

In [31]:
class MultiplyUserArgs(LLMPromptArgs):
    c: int


class MultiplyResponse(BaseModel):
    c_a_plus_b: int


multiply_in_prompt = (
    "Multiply {a_plus_b} by {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, None](
    name="multiply_agent",
    llm=OpenAILLM(model_name="openai:gpt-4.1"),
    usr_args_schema=MultiplyUserArgs,
    in_prompt=multiply_in_prompt,
    reset_memory_on_run=True,
)


@multiply_agent.parse_output
def parse_output(conversation: Messages, **kwargs) -> MultiplyResponse:
    return MultiplyResponse(c_a_plus_b=int(str(conversation[-1].content)))

In [32]:
seq_agent = SequentialWorkflow[AddInputArgs, MultiplyResponse, None](
    name="seq_agent", subprocs=[add_agent, multiply_agent]
)

In [33]:
add_run_args = RunArgs(usr=AddUserArgs(b=3))
multiply_run_args = RunArgs(usr=MultiplyUserArgs(c=6))

ctx = RunContext[None](
    run_args={"add_agent": add_run_args, "multiply_agent": multiply_run_args},
    print_messages=True,
)

In [34]:
# Can pass batched arguments
out = await seq_agent.run(in_args=[AddInputArgs(a=2)], ctx=ctx)
print_single_output(out)


<add_agent>[USER]
Add 2 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
5

------------------------------------
I/O/(R)/(C) tokens: 24/1/0/0

<multiply_agent>[USER]
Multiply 5 by 6. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
30

------------------------------------
I/O/(R)/(C) tokens: 25/1/0/0

<Response>
c_a_plus_b=30


# Agents as tools

When agents are used as tools, their `in_args` become the tool inputs.

This is how one can implement a manager + helpers architecture.

In [35]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5."
    ),
)

The JSON schema of `in_args` is preserved:

In [36]:
seq_tool.in_type.model_json_schema()

{'properties': {'a': {'description': 'First number to add.',
   'title': 'A',
   'type': 'integer'}},
 'required': ['a'],
 'title': 'AddInputArgs',
 'type': 'object'}

In [37]:
await seq_tool(a=15, ctx=ctx)


<add_agent>[USER]
Add 15 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
18

------------------------------------
I/O/(R)/(C) tokens: 24/1/0/0

<multiply_agent>[USER]
Multiply 18 by 6. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
108

------------------------------------
I/O/(R)/(C) tokens: 25/1/0/0


MultiplyResponse(c_a_plus_b=108)

# Teacher / students

A more advanced example of multi-agent debate, where agents communicate using the actor model.

In [38]:
def extract_recipients(message: str) -> list[str]:
    match = re.search(r"\[(.*?)\]", message)
    if match:
        # Extract the contents inside square brackets
        content = match.group(1)
        # Extract each student name within angle brackets
        return re.findall(r"<(.*?)>", content)  # Output: ['Alice', 'Bob', 'Charlie']

    return []

Communication schemas

In [39]:
# Teacher can choose which students to send the message to
# We need to inherit from DynCommPayload to use dynamic communication
class TeacherExplanation(DynCommPayload):
    explanation: str


# Students can only ask questions to the teacher
class StudentQuestion(BaseModel):
    question: str

In [40]:
pool = PacketPool[Any]()

#### Teacher

In [41]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    name="teacher",
    llm=OpenAILLM(model_name="openai:gpt-4o"),
    sys_prompt=teacher_sys_prompt,
    packet_pool=pool,
    # all available recipients to choose from:
    recipients=["student1", "student2"],
)


@teacher.parse_output
def parse_teacher_output(conversation: Messages, **kwargs) -> TeacherExplanation:
    message = str(conversation[-1].content)
    # Quick and dirty regexes to extract the recipients and explanation
    recipients = extract_recipients(message)
    explanation = message.split("[")[0].strip()

    # `selected_recipient_ids` is a required field for `DynCommPayload`
    return TeacherExplanation(explanation=explanation, selected_recipients=recipients)


@teacher.exit_communication
def teacher_exit_condition(out_packet: Packet[TeacherExplanation], ctx) -> bool:
    # Finish the conversation if the teacher says "Goodbye, students!"
    message = out_packet.payloads[0].explanation

    return "Goodbye, students!" in message

#### Students

In [42]:
student_sys_prompts = [
    """
You are a 4-year old child trying to make sense of physics. 
Your name is <student1>.
Talk to the teacher to understand the topic.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 4-year old child.
You talk to the teacher only.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    return LLMAgent[TeacherExplanation, StudentQuestion, None](
        name=name,
        llm=OpenAILLM(model_name="openai:gpt-4o"),
        sys_prompt=sys_prompt,
        packet_pool=pool,
        recipients=["teacher"],
    )


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])


@student1.parse_output
def parse_student1_output(conversation: Messages, **kwargs) -> StudentQuestion:
    return StudentQuestion(question="<student1>: " + str(conversation[-1].content))


@student2.parse_output
def parse_student2_output(conversation: Messages, **kwargs) -> StudentQuestion:
    return StudentQuestion(question="<student2>: " + str(conversation[-1].content))

Specify shared context 

In [43]:
ctx = RunContext[None](print_messages=True)
ctx.printer.color_by = "agent"

Run and wait until completion

In [ ]:
await teacher.start_listening(ctx=ctx)
await student1.start_listening(ctx=ctx)
await student2.start_listening(ctx=ctx)

# Teacher starts the conversation by posting a message to the pool
_ = await teacher.run("start", ctx=ctx)

# Custom AsyncOpenAI client

In [45]:
from openai import AsyncOpenAI
import os

# Custom client
client = AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="deepseek/deepseek-r1-0528",
        llm_settings=OpenAILLMSettings(logprobs=True),
        client=client,
    ),
)


ctx = RunContext[None](print_messages=True)
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)


<chatbot>[USER]
Output a list of 3 integers from 0 to 10 as a python array, no talking

<chatbot>[ASSISTANT]
```python
[3, 7, 9]
```

------------------------------------
I/O/(R)/(C) tokens: 23/110

<Response>
[3, 7, 9]
